In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "at010-agent-chat"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.

from typing import cast
import uuid

from agentex.types import Task

TASK_ID = str(uuid.uuid4())[:8]

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="task/create",
    params={
        "name": f"{TASK_ID}-task",
        "params": {}
    }
)

task = cast(Task, rpc_response.result)
print(task)

Task(id='4f06d705-24e8-4008-ad90-693072386cc5', created_at=datetime.datetime(2025, 7, 27, 6, 4, 37, 584907, tzinfo=TzInfo(UTC)), name='f282773f-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 27, 6, 4, 37, 584907, tzinfo=TzInfo(UTC)))


In [4]:
# Test non streaming response
from typing import cast
from agentex.types import Event

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="event/send",
    params={
        "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups"},
        "task_id": task.id,
    }
)

event = cast(Event, rpc_response.result)
print(event)

Event(id='ce9ba762-9db3-4779-906c-9acec68c1f9e', agent_id='04a102f4-984c-4deb-8402-b9d491074a4a', sequence_id=232, task_id='4f06d705-24e8-4008-ad90-693072386cc5', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 27, 6, 4, 38, 128727, tzinfo=TzInfo(UTC)))


In [ ]:
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=20,
)

╭───────────────────────── USER [07/27/2025 06:04:38] ─────────────────────────╮
│ Hello tell me the latest news about AI and AI startups                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 06:04:47] ─────────────────────────╮
│ 🔧 Tool Request: web_search                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "latest news about AI and AI startups",                          │
│    "model": "gpt-4o-mini",                                                   │
│    "type": "web_search_preview"                                              │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 06:04:47] ─────────────────────────╮
│ ✅ Tool Response: web_search                                                 │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "Artificial intelligence (AI) continues to be a dynamic and       │
│  rapidly evolving sector, with significant developments in both global       │
│  initiatives and startup activities.\n\n**Global AI Initiatives**\n\nChina   │
│  has proposed the establishment of a new international organization aimed a  │
│  promoting global cooperation in AI. This initiative seeks to provide an     │
│  alternative to U.S.-led efforts and foster inclusive development of AI      │
│  technologies. Premier Li Qiang announced the proposal at the World          │
│  Artificial Intelligence Conference in Shanghai, emphasizing equitable acce  │
│  to AI for all nations, particularly those in the Global South. The plan     │
│  includes hosting the organization's headquarters in Shanghai and involves   │
│  collaboration among governments, industry leaders, and academics worldwide  │
│  ([reuters.com](https://www.reuters.com/world/china/china-proposes-new-glob  │
│  -ai-cooperation-organisation-2025-07-26/?utm_source=openai))\n\n**AI Start  │
│  Funding and Developments**\n\nThe AI startup ecosystem is experiencing      │
│  unprecedented growth, with substantial investments and valuations:\n\n-     │
│  **Perplexity**: This AI search startup has achieved a valuation of $18      │
│  billion following a $100 million funding round, reflecting strong investor  │
│  confidence in the competitive AI search market.                             │
│  ([pymnts.com](https://www.pymnts.com/news/artificial-intelligence/2025/thi  │
│  week-in-ai-ai-startups-hit-fundraising-gold/?utm_source=openai))\n\n-       │
│  **Thinking Machines**: Founded by former OpenAI CTO Mira Murati, Thinking   │
│  Machines has raised $2 billion, valuing the startup at $10 billion. The     │
│  company plans to unveil its first product soon, featuring a significant     │
│  open-source component to support researchers and startups developing custo  │
│  models.                                                                     │
│  ([pymnts.com](https://www.pymnts.com/news/artificial-intelligence/2025/thi  │
│  week-in-ai-ai-startups-hit-fundraising-gold/?utm_source=openai))\n\n-       │
│  **Anthropic**: Reports indicate that Anthropic is being approached by       │
│  investors whose funding offers could value the startup at $100 billion,     │
│  highlighting the growing interest in AI safety and ethics.                  │
│  ([pymnts.com](https://www.pymnts.com/news/artificial-intelligence/2025/thi  │
│  week-in-ai-ai-startups-hit-fundraising-gold/?utm_source=openai))\n\n-       │
│  **Mistral AI**: The French AI startup, specializing in open-weight large    │
│  language models, secured \u20ac600 million in funding in June 2024,         │
│  elevating its valuation to \u20ac5.8 billion. This positions Mistral as a   │
│  significant player in the global AI race, particularly outside the San      │
│  Francisco Bay Area.                                                         │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Mistral_AI?utm_source=op  │
│  ai))\n\n- **Applied Intuition**: In June 2025, Applied Intuition achieved   │
│  $15 billion valuation after completing a $600 million Series F funding      │
│  round. The company specializes in AI software for autonomous systems and h  │
│  partnered with automotive man

╭──────────────────────── AGENT [07/27/2025 06:04:49] ─────────────────────────╮
│ Here's the latest news about AI and AI startups:                             │
│                                                                              │
│                            Global AI Initiatives                             │
│                                                                              │
│  • China's Proposal: China has proposed the creation of a new international  │
│    organization for AI cooperation. This aims to promote global              │
│    collaboration, with headquarters potentially in Shanghai. The goal is to  │
│    ensure equitable AI access and development, particularly benefiting       │
│    nations in the Global South. (]8;id=31401;https://www.reuters.com/world/china/china-proposes-new-global-ai-cooperation-organisation-2025-07-26/?utm_source=openai\Source]8;;\)                                     │
│                                                                              │
│                           AI Startup Developments                            │
│                                                                              │
│  • Perplexity: An AI search startup, Perplexity, is valued at $18 billion    │
│    after a $100 million funding round. (]8;id=615514;https://www.pymnts.com/news/artificial-intelligence/2025/this-week-in-ai-ai-startups-hit-fundraising-gold/?utm_source=openai\Source]8;;\)                              │
│  • Thinking Machines: Founded by former OpenAI CTO Mira Murati, this startup │
│    raised $2 billion, reaching a $10 billion valuation. It plans to launch   │
│    its first product with a significant open-source component. (]8;id=877122;https://www.pymnts.com/news/artificial-intelligence/2025/this-week-in-ai-ai-startups-hit-fundraising-gold/?utm_source=openai\Source]8;;\)      │
│  • Anthropic: Investors are reportedly interested in valuing this AI         │
│    safety-focused startup at $100 billion. (]8;id=634141;https://www.pymnts.com/news/artificial-intelligence/2025/this-week-in-ai-ai-startups-hit-fundraising-gold/?utm_source=openai\Source]8;;\)                          │
│  • Mistral AI: This French startup raised €600 million, achieving a          │
│    valuation of €5.8 billion. It focuses on open-weight large language       │
│    models. (]8;id=894816;https://en.wikipedia.org/wiki/Mistral_AI?utm_source=openai\Source]8;;\)                                                          │
│  • Applied Intuition: Specializing in AI for autonomous systems, it reached  │
│    a $15 billion valuation after a $600 million funding round. (]8;id=900026;https://en.wikipedia.org/wiki/Applied_Intuition?utm_source=openai\Source]8;;\)      │
│                                                                              │
│                            AI in Venture Capital                             │
│                                                                              │
│  • Investment Surge: In 2025, AI startups accounted for 53% of global        │
│    venture capital investments. However, more than a third of U.S. VC funds  │
│    were directed to just five companies. (]8;id=854031;https://www.axios.com/2025/07/03/ai-startups-vc-investments?utm_source=openai\Source]8;;\)                            │
│                                                                              │
│ These highlights reveal the dynamic growth and transformative impact of AI   │
│ across sectors globally.                                                     │
╰──────────────────────────────────────────────────────────────────────────────╯